# 0 Intro

In this notebook, we attempt to reproduce the [*Parking Tickets in Toronto*](https://schoolofcities.github.io/parking-tickets-toronto/) visualization by [Jeff Allan](https://schoolofcities.utoronto.ca/people/jeff-allen/). Jeff works at UofT's [School of Cities](https://schoolofcities.utoronto.ca/). He has produced many amazing [geospatial data visualizations](http://jamaps.github.io/maps.html).

I believe Jeff did this plot using [QGIS](https://www.qgis.org/en/site/) and [Inkscape](https://inkscape.org/), both are amazing open-source tools, one for GIS (Geographic Information System) and the other for drawing. Here, however, we will attempt to reproduce this plot using [Matplotlib](https://matplotlib.org/) and [GeoPandas](https://geopandas.org/en/stable/)' `plot()` method. The GeoPandas' `plot()` method is just a high-level interface to Matplotlib for making maps.

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd

In [ ]:
# check the versions of the three Python packages to be used
print(f"matplotlib version: {mpl.__version__}")
print(f"geopandas version: {gpd.__version__}")
print(f"pandas version: {pd.__version__}")

In [ ]:
# set the panadas option to display all table columns in this notebook
pd.set_option('display.max_columns', None)

# 1 Datasets

We will use three datasets.

1. Toronto parking ticket count dataset. This dataset is prepared by Jeff Allan. You can find how it's prepared at its Github repository [here](https://github.com/schoolofcities/parking-tickets-toronto) (see the data folder).

2. Toronto boundary data set. You can find this dataset [here](https://open.toronto.ca/dataset/regional-municipal-boundary/) at Toronto's Open Data portal.

3. Toronto Centreline dataset. You can find this dataset [here](https://open.toronto.ca/dataset/toronto-centreline-tcl/) at Toronto's Open Data portal. We are using the Version 2 data in GeoJSON format.

I have downloaded all the datasets and uploaded them in a Github repository for us to use. (I also zipped the original centreline `.geojson` file to reduce its size.)

Now, let's download all three datasets.

In [ ]:
# Download the datasets
!wget --quiet https://github.com/tdmdal/datasets-teaching/raw/main/ptickets/all.csv
!wget --quiet https://github.com/tdmdal/datasets-teaching/raw/main/ptickets/Centreline%20-%20Version%202.geojson.zip
!wget --quiet https://github.com/tdmdal/datasets-teaching/raw/main/ptickets/toronto-boundary-wgs84.zip

# 2 Load and prepare data

Let's load each dataset and take a quick look.

In [ ]:
# load ticket count data in csv format
ticket = pd.read_csv("all.csv")
ticket.head()

In [ ]:
# load zipped toronto boundary vector data in shape file format (together with
# associated attribute and index files)
boundary = gpd.read_file("zip://toronto-boundary-wgs84.zip")
boundary.head()

In [ ]:
# load zipped centreline data in geojson format
geojson_file = "zip://Centreline - Version 2.geojson.zip"
centreline = gpd.read_file(geojson_file)
centreline.head()

In [ ]:
# list unique features in the FEATURE_CODE_DESC column
centreline['FEATURE_CODE_DESC'].unique()

Let's merge the centreline data with the ticket count data, and extract the columns we needed for visualization.

In [ ]:
# merge centreline table with ticket count table
# select only columns needed for plotting
centreline = centreline[['CENTRELINE_ID', 'FEATURE_CODE_DESC', 'geometry']].merge(
    ticket[['CENTRELINE_ID', 'count_all']], how='left', on='CENTRELINE_ID')
centreline.head()

# 3 Create a default plot

In [ ]:
# draft a heatmap plot (a choropleth map)

# create figure and axes
fig, ax = plt.subplots()

# plot the boundary
boundary.plot(ax=ax)

# plot the centreline (with ticket count as color code)
centreline.plot(ax=ax, column='count_all', cmap='plasma')

In [ ]:
# plot the histogram of ticket count
# the distribution has a long tail
centreline['count_all'].hist()

In [ ]:
# plot the histogram of ticket count that is < 5000
centreline[centreline['count_all'] < 5000]["count_all"].hist(bins=50)

In [ ]:
# draft a heatmap plot (a choropleth map) with a subset of ticket count data

# use dark theme
plt.style.use('dark_background')

# create figure and axes
fig, ax = plt.subplots()

# don't show axis
ax.set_axis_off()

# plot the boundary
boundary.plot(ax=ax, edgecolor='gray', facecolor='none', markersize=1)

# plot the centreline
centreline[centreline['count_all'] < 5000].plot(ax=ax, column='count_all', cmap='plasma', markersize=1)

# 4 Fine tune the plot

## 4.1 Refine 1 - Better orientation

In [ ]:
# rotate centreline
# turn EPSG:4326 to EPSG:3347 first to avoid shape distortion after rotation
cline_3347 = centreline.to_crs(epsg=3347)

# rotate with respect to the centroid of all centrelines
cline_3347_rotated = cline_3347.rotate(-28, origin=cline_3347.unary_union.centroid).rename("geometry_3347_rotate")

# A geopandas GeoSeries is returned after rotation
cline_3347_rotated.head()

In [ ]:
# combine the original centreline GeoDataFrame with the rotated GeoSeries
centreline_rotated = centreline.join(cline_3347_rotated)

# pick only the needed columns
centreline_rotated = centreline_rotated[["CENTRELINE_ID", "FEATURE_CODE_DESC", "count_all", "geometry_3347_rotate"]].rename(columns={"geometry_3347_rotate": "geometry"})

# check the result
centreline_rotated.head()

In [ ]:
# rotate boundary
# turn EPSG:4326 to EPSG:3347 first to avoid shape distortion after rotation
# rotate with respect to the centroid of all centrelines to match centreline rotation centroid
boundary_rotated = boundary.to_crs(epsg=3347).rotate(-28, origin=cline_3347.unary_union.centroid)
boundary_rotated.head()

In [ ]:
# draft a heatmap plot (a choropleth map)

# use default theme
plt.style.use('default')

# create figure and axes
fig, ax = plt.subplots()

# plot the boundary
boundary_rotated.plot(ax=ax)

# plot the centreline (with ticket count as color code)
centreline_rotated.plot(ax=ax, column='count_all', cmap='plasma')

In [ ]:
# draft a heatmap plot (a choropleth map) with a subset of ticket count data

# use dark themem
plt.style.use('dark_background')

# create figure and axes
fig, ax = plt.subplots()

# don't show axis
ax.set_axis_off()

# plot the boundary
boundary_rotated.plot(ax=ax, edgecolor='gray', facecolor='none', markersize=1)

# plot the centreline
centreline_rotated[centreline_rotated['count_all'] < 5000].plot(ax=ax, column='count_all', cmap='plasma', markersize=1)

## 4.1 Refine 2 - Colormap on discrete intervals

In [ ]:
# turn off vertical scroll (for large images)
from google.colab import output
output.no_vertical_scroll()

# import a few additional functions form matplotlib
from matplotlib.colors import BoundaryNorm

# use dark themem
plt.style.use('dark_background')

# create figure and axes
fig, ax = plt.subplots(figsize=(15, 10), layout='constrained')

# don't show axis
ax.set_axis_off()

# plot the boundary
boundary_rotated.plot(ax=ax, edgecolor='gray', facecolor='none', markersize=1)

# Generate a colormap index based on discrete intervals
# https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.Colormap.html
# https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.BoundaryNorm.html
cmap = plt.colormaps['inferno'].with_extremes(over="white")
bounds = [5, 10, 25, 50, 100, 250, 500, 1000, 2500, 5000]
norm = BoundaryNorm(bounds, cmap.N, extend='both')

# plot centreline heatmap
centreline_rotated.plot(ax=ax, column='count_all',
                        cmap=cmap,
                        norm=norm,
                        markersize=0.5,
                        legend=True,
                        legend_kwds={
                            'shrink': 0.3,
                            'orientation': 'horizontal',
                            'pad': 0,
                            'anchor': (0.5, 1),
                            'extendfrac': 'auto',
                            'extendrect': True,
                            'label': 'Number of parking tickets per 100m'})

# add title and subtitle
fig.suptitle("Mapping Parking Tickets in Toronto", fontsize=25, fontweight='bold', color='yellow')
subtitle_text = ("Over 22.8 million parking tickets were issued in the City of Toronto in the \n"
                  "decade spanning 2011 to 2020, representing over 1 billion dollars in fines. \n"
                  "This map shows almost all of these parking tickets.")
_ = ax.set_title(subtitle_text, fontsize=16, y=1.05)

## 4.2 Refine 3 - A better colorbar

In [ ]:
# turn off vertical scroll (for large images)
from google.colab import output
output.no_vertical_scroll()

# import a few additional functions form matplotlib
from matplotlib.colors import BoundaryNorm
from matplotlib.cm import ScalarMappable

# use dark themem
plt.style.use('dark_background')

# create figure and axes
fig, ax = plt.subplots(figsize=(15, 10), layout='constrained')

# don't show axis
ax.set_axis_off()

# plot the boundary
boundary_rotated.plot(ax=ax, edgecolor='gray', facecolor='none', markersize=1)

# Generate a colormap index based on discrete intervals
# https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.Colormap.html
# https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.BoundaryNorm.html
cmap = plt.colormaps['inferno'].with_extremes(under='midnightblue', over='white')
bounds = [5, 10, 25, 50, 100, 250, 500, 1000, 2500, 5000]
norm = BoundaryNorm(bounds, cmap.N, extend='both')

# plot centreline heatmap
centreline_rotated.plot(ax=ax, column='count_all', cmap=cmap, norm=norm, markersize=0.5)

# plot coloarbar legend separately so as to customize its look
cbar = fig.colorbar(ScalarMappable(norm=norm, cmap=cmap),
                    ax=ax,
                    orientation='horizontal',
                    shrink=0.3,
                    pad=-0.02,
                    anchor=(0.5, 1),
                    extendfrac='auto',
                    extendrect=True,
                    drawedges=True,
                    label='Number of parking tickets per 100m')

cbar.ax.tick_params('x',
                    bottom=False, labelbottom=False,
                    top=True, labeltop=True,
                    labelrotation=45)

# add title and subtitle
fig.suptitle("Mapping Parking Tickets in Toronto", fontsize=25, fontweight='bold', color='yellow')
subtitle_text = ("Over 22.8 million parking tickets were issued in the City of Toronto in the \n"
                  "decade spanning 2011 to 2020, representing over 1 billion dollars in fines. \n"
                  "This map shows almost all of these parking tickets.")
_ = ax.set_title(subtitle_text,
                 fontsize=16,
                 y=1.04,
                 multialignment='left')


## 4.4 Refine 4 - Scalebar, north arrow & notes

In [ ]:
!pip install --quiet matplotlib-scalebar

In [ ]:
# turn off vertical scroll (for large images)
from google.colab import output
output.no_vertical_scroll()

# import a few additional functions form matplotlib
from matplotlib.colors import BoundaryNorm
from matplotlib.cm import ScalarMappable
from matplotlib_scalebar.scalebar import ScaleBar

# use dark themem
plt.style.use('dark_background')

# create figure and axes
fig, ax = plt.subplots(figsize=(15, 10), layout='constrained')

# don't show axis
ax.set_axis_off()

# plot the boundary
boundary_rotated.plot(ax=ax, edgecolor='gray', facecolor='none', markersize=1)

# Generate a colormap index based on discrete intervals
# https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.Colormap.html
# https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.BoundaryNorm.html
cmap = plt.colormaps['inferno'].with_extremes(under='midnightblue', over='white')
bounds = [5, 10, 25, 50, 100, 250, 500, 1000, 2500, 5000]
norm = BoundaryNorm(bounds, cmap.N, extend='both')

# exclude some centrelines from the plot
feature_exclude = ['Minor Arterial', 'Other', 'Hydro Line', 'Walkway',
                   'Minor Shoreline (Land locked)', 'Minor Railway', 'Trail',
                   'Access Road', 'Other Ramp', 'Minor Arterial Ramp']

# plot centreline heatmap
centreline_rotated[~centreline_rotated['FEATURE_CODE_DESC'].isin(feature_exclude)].plot(
    ax=ax, column='count_all', cmap=cmap, norm=norm, markersize=0.5)

# plot coloarbar legend separately so as to customize its look
cbar = fig.colorbar(ScalarMappable(norm=norm, cmap=cmap),
                    ax=ax,
                    orientation='horizontal',
                    shrink=0.23,
                    pad=-0.02,
                    anchor=(0.5, 1),
                    extendfrac='auto',
                    extendrect=True,
                    drawedges=True,
                    label='Number of parking tickets per 100m')

cbar.ax.tick_params('x',
                    bottom=False, labelbottom=False,
                    top=True, labeltop=True,
                    labelrotation=45)

# add title and subtitle
fig.suptitle("Mapping Parking Tickets in Toronto", fontsize=25, fontweight='bold', color='yellow')
subtitle_text = ("Over 22.8 million parking tickets were issued in the City of Toronto in the \n"
                  "decade spanning 2011 to 2020, representing over 1 billion dollars in fines. \n"
                  "This map shows almost all of these parking tickets.")
_ = ax.set_title(subtitle_text,
                 fontsize=16,
                 y=1.04,
                 multialignment='left')

# add scalebar
# https://geopandas.org/en/stable/gallery/matplotlib_scalebar.html
scale = ScaleBar(dx=1,
                 location='lower right',
                 color='grey',
                 box_alpha=0,
                 width_fraction=0.005,
                 border_pad=5)

_ = ax.add_artist(scale)

# add north arrow
# https://matplotlib.org/stable/users/explain/text/annotations.html
_ = ax.annotate("N",
                xy=(0.91, 0.25), xycoords='figure fraction',
                xytext=(0.9, 0.19), textcoords='figure fraction',
                ha='center',
                color='gray',
                arrowprops=dict(arrowstyle="fancy", color="gray"))

# add notes
_ = ax.text(0.6, 0.11,
            ("Note: This is an attempt to reproduce Jeff Allan's Toronto \nParking Tickets Map using Matplotlib. "
             "Find the original plot \nat https://schoolofcities.github.io/parking-tickets-toronto/.\n\n"
             "Data source and data processing code can be found therein."),
            transform=ax.transAxes,
            wrap=True,
            fontsize=8,
            horizontalalignment='left',
            bbox=dict(boxstyle='square', pad=1, facecolor='black', edgecolor='black'))